<a href="https://colab.research.google.com/github/juan-ivan-NV/Prices_Tracker/blob/main/Scraping_Functions.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Libraries & settings

In [ ]:
!pip install selenium
!apt-get update 
!apt install chromium-chromedriver
!pip install beautifulsoup4

from selenium import webdriver
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')
#chrome_options.add_argument('useAutomationExtention', False)
chrome_options.add_argument('--ifnore-certificate-errors')
chrome_options.add_argument('--start-maximized')

wd = webdriver.Chrome('chromedriver',chrome_options=chrome_options)

     |████████████████████████████████| 958 kB 4.3 MB/s 
     |████████████████████████████████| 356 kB 41.1 MB/s 
     |████████████████████████████████| 138 kB 48.6 MB/s 
     |████████████████████████████████| 3.6 MB 40.3 MB/s 
     |████████████████████████████████| 55 kB 3.2 MB/s 
     |████████████████████████████████| 54 kB 3.4 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
requests 2.23.0 requires urllib3!=1.25.0,!=1.25.1,<1.26,>=1.21.1, but you have urllib3 1.26.7 which is incompatible.
datascience 0.10.6 requires folium==0.2.1, but you have folium 0.8.3 which is incompatible.
Get:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:2 https://developer.down

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:15: DeprecationWarning: use options instead of chrome_options
  from ipykernel import kernelapp as app


In [ ]:
from selenium.common.exceptions import TimeoutException, NoSuchElementException
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support.ui import WebDriverWait, Select
from selenium.webdriver.firefox.options import Options
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from pandas.io.json import json_normalize
from bs4 import BeautifulSoup 
from time import sleep
from lxml import html
import pandas as pd 
import requests
import time
import json
import time
import os
import re


/usr/local/lib/python3.7/dist-packages/requests/__init__.py:91: RequestsDependencyWarning: urllib3 (1.26.7) or chardet (3.0.4) doesn't match a supported version!
  RequestsDependencyWarning)


## Global search

In [ ]:
search = 'playstation 5'

# Scrapping ebay


In [ ]:

def ebay_scraper(search_keyword):
  """Function to get some items info from ebay"""
  
  url = "https://www.ebay.com/"
  browser = wd
  browser.implicitly_wait(7)
  browser.get(url)

  #search_keyword = 'Playstation 5'
  search_field = browser.find_element_by_id('gh-ac')
  search_field.clear()
  search_field.send_keys(search_keyword)
  search_field.send_keys(Keys.ENTER)

  total_result = int(browser.find_element_by_class_name('srp-controls__count-heading').find_element_by_class_name('BOLD').text.replace(',', ''))
  ebay_listings = []

  current_page = 1
  to_continue = True

  while to_continue:
      try:
          #print('Processing page {0}'.format(current_page))

          soup = BeautifulSoup(browser.page_source, 'html.parser')
          item_list = soup.find_all('li', 's-item--watch-at-corner')
          
          for listing in item_list:
              product_detail = {}

              product_detail['product title'] = listing.h3.text
              product_detail['product url'] = listing.a['href']
              
              listing_subtitles = listing.find_all('div', 's-item__subtitle')
              if len(listing_subtitles) == 2:
                  listing_subtitle_1 = listing_subtitles[0].text
                  listing_subtitle_2 = listing_subtitles[1].text
              elif len(listing_subtitles) == 1:
                  listing_subtitle_1 = listing_subtitles[0].text
                  listing_subtitle_2 = None
              else:
                  listing_subtitle_1 = None
                  listing_subtitle_2 = None

              product_detail['subtitle1'] = listing_subtitle_1
              product_detail['subtitle2'] = listing_subtitle_2


              # Star rating and product review count
              try:
                  x_star_rating = listing.find('div', 'x-star-rating')
                  stars = len(x_star_rating.find_all('svg', 'icon--star-full-small'))
                  if int(stars) > 4: print(stars) 
                  review_count = int(listing.find('span', 's-item__reviews-count').span.text.replace( ' product ratings', ''))
                  
                  product_detail['stars'] = stars
                  product_detail['product review_count'] = review_count
              except Exception as e:
                  product_detail['stars'] = ''
                  product_detail['product review_count'] = ''

              product_detail['price'] = listing.find('span', 's-item__price').text

              try:
                  product_detail['shipping detail'] = listing.find('span', 's-item__logisticsCost').text
              except AttributeError:
                  product_detail['shipping detail'] = 'Not Available'
                  
              ebay_listings.append(product_detail)


          time.sleep(1)
          browser.find_element_by_xpath('//a[@class="pagination__item" and text()="{0}"]'.format(current_page+1)).click()
          current_page +=1

          if current_page > 5:
              raise Exception('Stop')

      except NoSuchElementException:
          print('Last page {0}'.format(current_page))
          to_continue = False
  
  return pd.DataFrame(ebay_listings)


In [ ]:
ebay_df = ebay_scraper(search)
ebay_df = df[['product title', 'price', 'product review_count', 'product url']]
#ebay_df.sort_values(by='amount')
ebay_df.head()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:11: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  # This is added back by InteractiveShellApp.init_path()
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:16: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  app.launch_new_instance()
/usr/local/lib/python3.7/dist-packages/selenium/webdriver/remote/webelement.py:445: UserWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  warnings.warn("find_element_by_* commands are deprecated. Please use find_element() instead")


Processing page 1
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:74: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead


Processing page 2
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
Processing page 3
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
Processing page 4
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
Processing page 5
Last page 5


In [ ]:
print(df.iloc[103])

product title                      Playstation 5 Digital w/ 2 Controllers
product url             https://www.ebay.com/itm/234285013150?hash=ite...
price                                                             $660.00
product review_count                                                     
Name: 103, dtype: object


In [ ]:
#df.to_excel('playstation 5 listings.xlsx', index=False)


product title           PS5 Sony Playstation 5 Console Disc Version 🔥B...
product url             https://www.ebay.com/itm/304128005531?epid=190...
subtitle1               Brand New · Sony PlayStation 5 · Sony PlayStat...
subtitle2                                                            None
stars                                                                   0
product review_count                                                 1976
price                                                           $1,199.99
shipping detail                                             Free shipping
amount                                                             119999
Name: 103, dtype: object


In [ ]:
# https://towardsdatascience.com/scraping-multiple-amazon-stores-with-python-5eab811453a8

# scrapping Amazon



In [ ]:
# getting the next page
def get_urls(search_term):
  """Genetare the Url from search term"""
  template = "https://www.amazon.com.mx/s?k={}&__mk_es_MX=%C3%85M%C3%85%C5%BD%C3%95%C3%91&ref=nb_sb_noss_1"
  search_term = search_term.replace(' ', '+')

  # add term query to url
  url = template.format(search_term)

  # add page query placeholder
  url += '&page={}'

  return template.format(search_term)

# error handling
def extract_record(item):

  # description and url
  atag = item.h2.a
  description = atag.text.strip()
  url = 'https://www.amazon.com.mx' + atag.get('href')

  try:
    # price
    price_parent = item.find('span', 'a-price')
    price = price_parent.find('span', 'a-offscreen').text
  except AttributeError:
    return

  try:
    # reviews and rating
    rating = item.i.text
    #review_count = item.find('span', {'class':'a-size-base'}).text
    review_count = item.find('div', {'class':'a-section a-spacing-none a-spacing-top-micro'}).find('span', class_= 'a-size-base').text
  except AttributeError:
    rating = ''
    review_count = ''

  result = (description, price, rating, review_count, url)

  return result

def amazon_scraper(search_term):
  """run main program"""
  # set webdrriver if needed

  records = []
  url = get_urls(search_term)

  for page in range(1,6):
    wd.get(url.format(page))
    soup = BeautifulSoup(wd.page_source, 'html.parser')
    results = soup.find_all('div', {'data-component-type': 's-search-result'})

    for item in results:
      record = extract_record(item)
      if record:
        records.append(record)
  
  wd.close()

  return pd.DataFrame(records, columns = ['Description', 'Price', 'Rating', 'Reviews', 'Url'])

In [ ]:
amazon_df = amazon_scraper(search)
amazon_df = amazon_df[['Description','Price','Reviews','Url']]

In [ ]:
amazon_df.head(10)

,Description,Price,Reviews,Url
0,Consola PS5 Digital - Standard Edition,"$11,499.00",,https://www.amazon.com.mx/Consola-PS5-Digital-...
1,Control Inalámbrico PS5 DualSense Midnight Black,"$1,349.00","3,497",https://www.amazon.com.mx/Control-Inal%C3%A1mb...
2,PULSE 3D Wireless Headset - PULSE 3D Edition,"$2,045.95","7,419",https://www.amazon.com.mx/PULSE-3D-Wireless-He...
3,Sony PlayStation 5 Dualsense - Control Inalámb...,"$1,199.00","3,497",https://www.amazon.com.mx/Sony-Interactive-Ent...
4,Control Inalámbrico PS5 DualSense Cosmic Red,"$1,349.00","3,497",https://www.amazon.com.mx/Control-Inal%C3%A1mb...
5,Scarlet Nexus - Standard Edition - Playstation 5,$719.00,48,https://www.amazon.com.mx/Scarlet-Nexus-Standa...
6,Ventilador de refrigeración PS5 con soporte pa...,$799.00,24,https://www.amazon.com.mx/Ventilador-refrigera...
7,Gran Turismo 7 - 25th Anniversary Edition - Pl...,"$2,299.00",,https://www.amazon.com.mx/Gran-Turismo-25th-An...
8,Hot Wheels Unleashed - Standard Edition - Play...,$631.04,21,https://www.amazon.com.mx/Hot-Wheels-Unleashed...
9,Deathloop for PlayStation 5,$788.85,11,https://www.amazon.com.mx/DE1CSTP5PENA-Deathlo...


# Scrapping mercado libre

In [ ]:
# getting the next page
def get_urls(search_term):
  """Genetare the Url from search term"""
  template = "https://listado.mercadolibre.com.mx/{}"
  search_term = search_term.replace(' ', '-')

  # add term query to url
  url = template.format(search_term)

  # add page query placeholder
  url += '_Desde_{}_NoIndex_True'

  return template.format(search_term)

# error handling
def extract_record(item):

  # description and url
  description = item.h2.text.strip()
  url = item.a['href']

  try:
    # price
    price_parent = item.find('div', {'class':'ui-search-price__second-line'})
    price = price_parent.find('span', 'price-tag-amount').text
  except AttributeError:
    return ''

  try:
    # reviews and rating
    stars = len(item.find_all('svg', {'class':'ui-search-icon ui-search-icon--star ui-search-icon--star-full'}))
    reviews = item.find('span', {'class':'ui-search-reviews__amount'}).text
  except AttributeError:
    stars = ''
    reviews = ''

  result = (description, price, stars, reviews, url)

  return result

def mercado_scraper(search_term):
  """run maun program"""
  # set webdrriver if needed

  records = []
  url = get_urls(search_term)

  items_pp = 1
  to_continue = True
  
  while to_continue:

    try:
      wd.get(url.format(items_pp))
      soup = BeautifulSoup(wd.page_source, 'html.parser')
      results = soup.find_all('li', {'class': 'ui-search-layout__item'})

      for item in results:
        record = extract_record(item)
      
        if record:
          records.append(record)
      items_pp += 50 
    
    except NoSuchElementException:
      print('Last page {0}'.format(current_page))
      to_continue = False

    if items_pp >= 151: to_continue = False 

  wd.close()

  return pd.DataFrame(records, columns = ['Description', 'Price', 'Rating', 'Reviews', 'Url'])

In [ ]:
mercado_libre_df = mercado_scraper(search)

In [ ]:
mercado_libre_df.head(10)

,Description,Price,Rating,Reviews,Url
0,Sony PlayStation 5 825GB Standard color blanc...,"$13,999",4,469,https://www.mercadolibre.com.mx/sony-playstati...
1,Sony PlayStation 5 825GB Digital Edition color...,"$11,499",4,104,https://www.mercadolibre.com.mx/sony-playstati...
2,Consola Retro Premium Gamebox Android Hdmi Con...,"$1,899",,,https://articulo.mercadolibre.com.mx/MLM-13410...
3,Playstation 4 Con Lector De Discos + 5 Juegos ...,"$10,990",,,https://articulo.mercadolibre.com.mx/MLM-77035...
4,Consola Playstation 5 825gb,"$15,999",,,https://articulo.mercadolibre.com.mx/MLM-89698...
5,Sony Playstation 5 825gb Standard Color Blanc...,"$17,969",,,https://articulo.mercadolibre.com.mx/MLM-91737...
6,Play Station 5 Nuevo Sellado Original,"$18,499",,,https://articulo.mercadolibre.com.mx/MLM-85228...
7,Consola Playstation 5 Ps5 Standard 825gb,"$22,999",,,https://articulo.mercadolibre.com.mx/MLM-86779...
8,Ps5 Consola Playstation 5 825gb Standard Color...,"$17,999",,,https://articulo.mercadolibre.com.mx/MLM-89813...
9,Consola Playstation 5 Standard Edition,"$16,789",,,https://articulo.mercadolibre.com.mx/MLM-83871...


# Scrapping wallmart mx

In [ ]:


headers = {
    'Accept': 'application/json',
    'Referer': 'https://www.walmart.com.mx/productos?Ntt=xiaomi&page=0',
    'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/94.0.4606.81 Safari/537.36',
    'Content-Type': 'application/json',
    'Cookie': 'Cookie_1=value; TS014f2e15=01538efd7c027ce77e369e2bc6d77f0399abd465a588ddf5af91f7df97ebd11d0588faf61dd6c5ce95485742c700a31dd976d66ee4; akavpau_vp_walmart_ss=1634878568~id=8dbbff347a90a0bea31a88fa333da0d1'
  }

wallmart_df = pd.DataFrame([])

for page in range(1,5):
  search = search.replace(' ','-')
  url = f"https://www.walmart.com.mx/api/v2/page/search?Ntt={search}&page={page}&size=48"
  r = requests.request("GET", url, headers=headers)
  data = json.loads(r.text)
  wallmart_df = wallmart_df.append(json_normalize(data['appendix']['SearchResults']['content']), ignore_index=True)
  time.sleep(4)
  print(f'Getting page {page} ...')


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:18: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead


Getting page 1 ...
Getting page 2 ...
Getting page 3 ...
Getting page 4 ...


In [ ]:
print(len(wallmart_df))

192


In [ ]:
wallmart_df.head()


,id,productId,skuDisplayName,productSeoUrl,productRatings,weighable,freeShippingItem,skuPrice,bigItem,onDemand,bundle,preOrderable,pickUpEnabled,brandName,score,salesRank,variants,imageUrls.small,imageUrls.large,variantGroupId
0,00081429001723,00081429001723,WRC 10 - PlayStation 5 Sony PlayStation 5,/videojuegos/playstation-5/juegos-ps5/wrc-10-p...,0.0,False,No Free Shipping,1399.0,False,False,False,False,True,sony,38.345955,0.0,[],/gm/3pp/asr/41d028f2-5b92-4725-b4ea-94d3c54666...,/gm/3pp/asr/41d028f2-5b92-4725-b4ea-94d3c54666...,NaN
1,00081287201230,00081287201230,Ghostrunner - PlayStation 5 Sony PlayStation 5,/videojuegos/playstation-5/juegos-ps5/ghostrun...,0.0,False,No Free Shipping,1128.0,False,False,False,False,True,sony,40.057320,0.0,[],/gm/3pp/asr/44be55ea-29f8-46a9-bf9b-a622fc2522...,/gm/3pp/asr/44be55ea-29f8-46a9-bf9b-a622fc2522...,NaN
2,00081933502095,00081933502095,Aragami 2 - PlayStation 5 Sony PlayStation 5,/videojuegos/playstation-5/juegos-ps5/aragami-...,0.0,False,No Free Shipping,1399.0,False,False,False,False,True,sony,38.345955,0.0,[],/gm/3pp/asr/6f34a629-c8f1-4b89-92bf-906ea11392...,/gm/3pp/asr/6f34a629-c8f1-4b89-92bf-906ea11392...,NaN
3,00088725611037,00088725611037,Far Cry 6 Standard Edition - PlayStation 4 Pla...,/videojuegos/playstation-5/juegos-ps5/far-cry-...,0.0,False,No Free Shipping,1349.0,False,False,False,False,True,sony,34.822571,0.0,[],/gm/3pp/asr/af7ee0bb-d40b-4cb8-86b5-d99db9ffc7...,/gm/3pp/asr/af7ee0bb-d40b-4cb8-86b5-d99db9ffc7...,NaN
4,00081002303723,00081002303723,Void Terrarium++ Deluxe Edition - PlayStation ...,/videojuegos/playstation-5/juegos-ps5/void-ter...,0.0,False,No Free Shipping,1199.0,False,False,False,False,True,sony,35.327385,0.0,[],/gm/3pp/asr/9b267aaa-c34c-49db-82de-47d1509960...,/gm/3pp/asr/9b267aaa-c34c-49db-82de-47d1509960...,NaN


# scrapping sams

In [ ]:

search = search.replace(' ','-')

url = f"https://www.sams.com.mx/sams/search?Ntt={search}&No=95"

payload={}
headers = {
  'authority': 'www.sams.com.mx',
  'accept': 'application/json, text/javascript, */*; q=0.01',
  'user-agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/94.0.4606.81 Safari/537.36',
  'x-requested-with': 'XMLHttpRequest',
  'sec-gpc': '1',
  'sec-fetch-site': 'same-origin',
  'sec-fetch-mode': 'cors',
  'sec-fetch-dest': 'empty',
  'referer': f'https://www.sams.com.mx/search/Ntt={search}',
  'accept-language': 'es-419,es;q=0.9',
  'cookie': 'CLOUD=prod-dfw6; JSESSIONID=h2MCuJH-5QoJQlr1FqYDmMVo.restapp-267254238-3-1340057250; AskZipcode=Y; CL=prod-cdc03; TS01f4281b=0130aff232911a4d804ff0629a31fa7bdfb057a225b7a0751bc1b5870768fd2ecc4571c25bd7e0702a120c39026ec0f9ab40a7f822; TS01e4bdf1=0130aff232911a4d804ff0629a31fa7bdfb057a225b7a0751bc1b5870768fd2ecc4571c25bd7e0702a120c39026ec0f9ab40a7f822; akavpau_vp_sams=1635024668~id=a7e5df36acebdff0445b69a9d1703bf3; akavpau_vp_sams=1635024947~id=c1b3151c319a92779e866250aacb2bce'
}

response = requests.request("GET", url, headers=headers)
soup = BeautifulSoup(response.content, 'lxml')


In [ ]:
sams_df = json_normalize(json.loads(soup.text)['mainArea'][10]['contents'][0]['records'])
sams_df.head()

JSONDecodeError: ignored

In [ ]:
sams_df.shape

(32, 35)

# scraping Ali express

In [ ]:
#Ali express data frame

def get_url(search, number):
  """Genetare the Url from search term"""
  search = search.replace(' ','+')
  myUrl = f'https://es.aliexpress.com/wholesale?trafficChannel=main&d=y&CatId=0&SearchText=={search}'
  template = myUrl + f'&ltype=wholesale&SortType=default&page={number}'
  return template

def aliexpress_scraper(search):
  """run main program"""
  # set webdrriver if needed

  df_aliexpress = pd.DataFrame(columns = ['description', 'price', 'link', 'sold', 'store','shipping'])
  titles = []
  prices = []
  links = []
  sold = []
  stores = []
  shippings = []

  for i in range(1,3):
    
    url = get_url(search, i) 
    wd.get(url)
    scrolled = 0
    scroll_size = 400

    while scrolled < 7:
        wd.execute_script('window.scrollTo(0, arguments[0]);', scroll_size)
        scrolled +=1
        scroll_size += 400
        time.sleep(1)

    # get html content
    tree = html.fromstring(wd.page_source)
    
    titles += tree.xpath('//div[@class="JIIxO"]//h1[@class="_3eC3x"]/text()')
    prices += re.findall('[^€]+€', ''.join(tree.xpath('//div[@class="JIIxO"]//div[@class="_13_ga _37W_B"]//span/text()')))
    links += [('https://es.aliexpress.com' + i) for i in tree.xpath('//div[@class="JIIxO"]//a[@class="_3KNwG _2f4Ho"]/@href')]
    sold += tree.xpath('//div[@class="JIIxO"]//span[@class="_2tU1w"]/text()')
    stores += [('https:' + i) for i in tree.xpath('//div[@class="JIIxO"]//a[@role="store"]/@href')]
    shippings += tree.xpath('//div[@class="JIIxO"]//span[@class="_1XYdp"]/text()')

  products = list(zip(titles, prices, links, sold, stores, shippings))

  df_aliexpress = df_aliexpress.append(products, ignore_index = True)

  return df_aliexpress

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:19: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead


In [ ]:
aliexpress_df = aliexpress_scraper(search)
aliexpress_df.head()

,0,1,2,3,4,5,description,link,price,shipping,sold,store
0,"Apple-teléfono móvil iPhone 11 Original, smart...",428.53€,https://es.aliexpress.com/item/100500332370564...,74 vendido(s),https://es.aliexpress.com/store/5324008,Envío gratis,NaN,NaN,NaN,NaN,NaN,NaN
1,Apple-teléfono inteligente iPhone 11 Original ...,448.87€,https://es.aliexpress.com/item/100500151912939...,43 vendido(s),https://es.aliexpress.com/store/1110648,"+ envío: € 92,28",NaN,NaN,NaN,NaN,NaN,NaN
2,"Apple-teléfono inteligente iPhone 11 Pro, Orig...",575.82€,https://es.aliexpress.com/item/100500245355153...,75 vendido(s),https://es.aliexpress.com/store/5324008,Envío gratis,NaN,NaN,NaN,NaN,NaN,NaN
3,"Apple-Smartphone iPhone 11 pro desbloqueado, t...",575.44€,https://es.aliexpress.com/item/100500266456325...,87 vendido(s),https://es.aliexpress.com/store/911615234,"+ envío: € 13,84",NaN,NaN,NaN,NaN,NaN,NaN
4,Apple-teléfono inteligente iPhone 11 desbloque...,437.47€,https://es.aliexpress.com/item/100500340290627...,18 vendido(s),https://es.aliexpress.com/store/1110648,"+ envío: € 92,28",NaN,NaN,NaN,NaN,NaN,NaN
5,"Apple-teléfono inteligente iPhone 11 pro max, ...",666.15€,https://es.aliexpress.com/item/100500305711728...,53 vendido(s),https://es.aliexpress.com/store/911615234,"+ envío: € 13,84",NaN,NaN,NaN,NaN,NaN,NaN
6,"Apple-teléfono inteligente iPhone 11 4G LTE, O...",417.31€,https://es.aliexpress.com/item/100500294000979...,25 vendido(s),https://es.aliexpress.com/store/709663,"+ envío: € 46,14",NaN,NaN,NaN,NaN,NaN,NaN
7,"Apple-teléfono inteligente iPhone 11, Original...",459.72€,https://es.aliexpress.com/item/100500261641230...,69 vendido(s),https://es.aliexpress.com/store/911615234,"+ envío: € 13,84",NaN,NaN,NaN,NaN,NaN,NaN
8,"Apple-teléfono inteligente iPhone 11 Pro Max, ...",666.11€,https://es.aliexpress.com/item/100500319950705...,52 vendido(s),https://es.aliexpress.com/store/911614417,Envío gratis,NaN,NaN,NaN,NaN,NaN,NaN
9,Apple-teléfono inteligente iPhone 11 desbloque...,482.18€,https://es.aliexpress.com/item/100500263723176...,46 vendido(s),https://es.aliexpress.com/store/1053035,Envío gratis,NaN,NaN,NaN,NaN,NaN,NaN


# scrapping copel

In [ ]:
# script completed for coppel

def get_url(search, number):
  """Genetare the Url from search term"""
  
  number = str(number)
  if str(number) == '0': number = '01' 
  search = search.replace(' ', '+')
  template = f"https://www.coppel.com/SearchDisplay?sType=SimpleSearch&ddkey=ProductListingView_6_-3011_1208&fromPage=catalogEntryList&searchTerm=i{search}&urlRequestType=Base&catalogId=10051&resultType=products&disableProductCompare=false&showResultsPage=true&beginIndex={number}&resultCatEntryType=2&langId=-5&enableSKUListView=false&pageSize=1&ajaxStoreImageDir=%2Fwcsstore%2FCoppelStoreFrontAssetStore%2F&storeId=10151"
  print(template)
  return template

  
def extract_record(item):
  """scrapping each item"""

  # description and url
  try:
    description = item.find('div', {'class':'product_name'}).find('a').text
    url = item.find('div', {'class':'product_name'}).find('a').get('href')
  except AttributeError:
    description = 'invalid item'
    url = 'empty'

  try:
    # price
    price = item.find('span',{'class':'unique_price'}).text
  except AttributeError:
    price = 'empty'

  result = (description, price, url)

  return result


def coppel_scraper(search):
  """run main program"""
  # set webdrriver if needed

  number = 0
  keep_going = True
  records = []
  #url = get_urls(search_term)

  while True:

    if number >= 37: break
    wd.get(get_url(search, number))
    soup = BeautifulSoup(wd.page_source, 'html.parser')
    results = soup.find_all('div', {'class': 'product'})
    print(len(results))

    for item in results:
      record = extract_record(item)
      if record:
        records.append(record)

    number += 12

  wd.close()
    
  return pd.DataFrame(records, columns = ['Description', 'Price', 'Url'])

In [ ]:
coppel_df = coppel_scraper(search)

https://www.coppel.com/SearchDisplay?sType=SimpleSearch&ddkey=ProductListingView_6_-3011_1208&fromPage=catalogEntryList&searchTerm=iiphone+13&urlRequestType=Base&catalogId=10051&resultType=products&disableProductCompare=false&showResultsPage=true&beginIndex=01&resultCatEntryType=2&langId=-5&enableSKUListView=false&pageSize=1&ajaxStoreImageDir=%2Fwcsstore%2FCoppelStoreFrontAssetStore%2F&storeId=10151
13
https://www.coppel.com/SearchDisplay?sType=SimpleSearch&ddkey=ProductListingView_6_-3011_1208&fromPage=catalogEntryList&searchTerm=iiphone+13&urlRequestType=Base&catalogId=10051&resultType=products&disableProductCompare=false&showResultsPage=true&beginIndex=12&resultCatEntryType=2&langId=-5&enableSKUListView=false&pageSize=1&ajaxStoreImageDir=%2Fwcsstore%2FCoppelStoreFrontAssetStore%2F&storeId=10151
13
https://www.coppel.com/SearchDisplay?sType=SimpleSearch&ddkey=ProductListingView_6_-3011_1208&fromPage=catalogEntryList&searchTerm=iiphone+13&urlRequestType=Base&catalogId=10051&resultType

In [ ]:
coppel_df.head(10)

,Description,Price,Url
0,\nTelcel Apple iPhone 13 mini 128 GB Pink\n,"\n\t\t\t\t$19,049\n\t\t\t",https://www.coppel.com/telcel-apple-iphone-13-...
1,\nTelcel Apple iPhone 13 mini 128 GB Starlight\n,"\n\t\t\t\t$19,049\n\t\t\t",https://www.coppel.com/telcel-apple-iphone-13-...
2,\nTelcel Apple 13 mini 128 GB (PRODUCT)RED\n,"\n\t\t\t\t$19,049\n\t\t\t",https://www.coppel.com/telcel-apple-13-mini-12...
3,\nApple Watch Series 7 45 mm Azul medianoche\n,"\n\t\t\t\t$15,399\n\t\t\t",https://www.coppel.com/apple-watch-series-7-45...
4,\nTelcel Apple iPhone 13 mini 256 GB Starlight\n,"\n\t\t\t\t$22,299\n\t\t\t",https://www.coppel.com/telcel-apple-iphone-13-...
5,\nTelcel Apple iPhone 13 mini 256 GB Midnight ...,"\n\t\t\t\t$22,299\n\t\t\t",https://www.coppel.com/telcel-apple-iphone-13-...
6,\nTelcel Apple iPhone 13 mini 256 GB Blue\n,"\n\t\t\t\t$22,299\n\t\t\t",https://www.coppel.com/telcel-apple-iphone-13-...
7,\nApple Watch Series 7 45 mm Verde\n,"\n\t\t\t\t$15,399\n\t\t\t",https://www.coppel.com/apple-watch-series-7-45...
8,\nTelcel Apple iPhone 13 128 GB Blue\n,"\n\t\t\t\t$22,299\n\t\t\t",https://www.coppel.com/telcel-apple-iphone-13-...
9,\nApple Watch Series 7 41 mm Azul medianoche\n,"\n\t\t\t\t$14,499\n\t\t\t",https://www.coppel.com/apple-watch-series-7-41...


# scrapping elektra

In [ ]:
def get_url(search, number):
  """Genetare the Url from search term"""

  search = search.replace(' ', '%20')
  template = f"https://www.elektra.com.mx/{search}?_q={search}&map=ft&page={str(number)}"
  return template


def elektra_scraper(search):
  """run main program"""
  # set webdrriver if needed

  #search = 'pantalla smart'
  items = []
  titles = []
  prices = []
  links = []

  for i in range(1,4):
    
    url = get_url(search, i) 
    wd.get(url)
    scrolled = 0
    scroll_size = 400

    while scrolled < 7:
        wd.execute_script('window.scrollTo(0, arguments[0]);', scroll_size)
        scrolled +=1
        scroll_size += 400
        time.sleep(1)

    xitems = wd.find_elements_by_xpath("//article[@class='vtex-product-summary-2-x-element pointer pt3 pb4 flex flex-column h-100']")
    items += [i.text for i in xitems]
    
    xtitles = wd.find_elements_by_xpath("//div[@class='vtex-search-result-3-x-galleryItem vtex-search-result-3-x-galleryItem--small pa4']\
                                       //span[@class='vtex-product-summary-2-x-productBrand vtex-product-summary-2-x-brandName t-body']")
    titles += [i.text for i in xtitles]
    
    xprices = wd.find_elements_by_class_name('vtex-store-components-3-x-price_sellingPrice--summary')
    prices += [i.text for i in xprices]
    
    xlinks = [elem.get_attribute('href') for elem in wd.find_elements_by_xpath("//a[@class='vtex-product-summary-2-x-clearLink h-100 flex flex-column']")]
    links += xlinks

  wd.close()
    
  return pd.DataFrame({'items':items, 
                       'titles':titles,
                       'prices':prices,
                       'links':links})

In [ ]:
elektra_df = elektra_scraper('pantalla smart')

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:32: DeprecationWarning: find_elements_by_* commands are deprecated. Please use find_elements() instead
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:35: DeprecationWarning: find_elements_by_* commands are deprecated. Please use find_elements() instead
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:38: DeprecationWarning: find_elements_by_* commands are deprecated. Please use find_elements() instead
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:41: DeprecationWarning: find_elements_by_* commands are deprecated. Please use find_elements() instead


In [ ]:
elektra_df.head()

,items,titles,prices,links
0,-\n70 %\n🔥 ÚLTIMAS PIEZAS\nPantalla OLED LG 65...,Pantalla OLED LG 65 Pulgadas 4K Smart OLED65E7...,"$48,900",https://www.elektra.com.mx/pantalla-oled-lg-65...
1,-\n25 %\nPantalla LED HKPro 32'' HD Smart TV H...,Pantalla LED HKPro 32'' HD Smart TV HKP32SM7,"$4,499",https://www.elektra.com.mx/pantalla-tv-led-32-...
2,-\n24 %\nSmart TV 32 Hyundai LED HD Linux HDMI...,Smart TV 32 Hyundai LED HD Linux HDMI USB HYLE...,"$5,599",https://www.elektra.com.mx/smart-tv-32-hyundai...
3,-\n13 %\n🔥 ÚLTIMAS PIEZAS\nPantalla LED Sony 3...,Pantalla LED Sony 32 Pulgadas HD Smart 32W600D,"$7,199",https://www.elektra.com.mx/sony-pantalla-32-pu...
4,-\n9 %\nSmart TV 32 Hyundai HD Linux Dolby Blu...,Smart TV 32 Hyundai HD Linux Dolby Bluetooth H...,"$5,600",https://www.elektra.com.mx/smart-tv-32-hyundai...


In [ ]:
print(len(elektra_df))

59


# Final data frame

In [ ]:
# data frames

ebay_df 
amazon_df
mercado_libre_df
wallmart_df
aliexpress_df
coppel_df
elektra_df